In [1]:
from tensorflow import keras

# Train all except the first two conv2d layers

In [36]:
vtid = keras.models.load_model('models/vtid2_1.h5')
vtid.trainable = False

for layer in vtid.layers[5:]:
  layer.trainable = True

x = vtid.layers[-2].output
outputs = keras.layers.Dense(6, activation='softmax', name='classifier')(x)
model = keras.Model(vtid.inputs, outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 128)       738

In [19]:
IMAGE_SIZE = 100
BATCH_SIZE = 32

train = keras.utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/train/", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
val = keras.utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/val", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)
test = keras.utils.image_dataset_from_directory("datasets/data/split/vehicles_zenodo/test", image_size=(IMAGE_SIZE, IMAGE_SIZE), batch_size=BATCH_SIZE)

callbacks_list = [
    # keras.callbacks.EarlyStopping(
    #     monitor="val_accuracy",
    #     patience=7
    # ),
    keras.callbacks.ModelCheckpoint(
        filepath="models/transfer_1.h5",
        monitor="val_loss",
        save_best_only=True
    )
]

history = model.fit(
  train.cache(),
  epochs=40,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Found 3360 files belonging to 6 classes.
Found 720 files belonging to 6 classes.
Found 720 files belonging to 6 classes.
Epoch 1/40


2022-08-02 06:19:52.933020: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


104/105 [============================>.] - ETA: 0s - loss: 1.4717 - accuracy: 0.4570

2022-08-02 06:19:56.473166: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 4s 36ms/step - loss: 1.4660 - accuracy: 0.4583 - val_loss: 0.9744 - val_accuracy: 0.6444
Epoch 2/40
105/105 [==============================] - 3s 29ms/step - loss: 0.8473 - accuracy: 0.6896 - val_loss: 1.1537 - val_accuracy: 0.6472
Epoch 3/40
105/105 [==============================] - 3s 28ms/step - loss: 0.6028 - accuracy: 0.7878 - val_loss: 0.5300 - val_accuracy: 0.8083
Epoch 4/40
105/105 [==============================] - 3s 29ms/step - loss: 0.4382 - accuracy: 0.8461 - val_loss: 0.6055 - val_accuracy: 0.8042
Epoch 5/40
105/105 [==============================] - 4s 34ms/step - loss: 0.3394 - accuracy: 0.8851 - val_loss: 0.5411 - val_accuracy: 0.8444
Epoch 6/40
105/105 [==============================] - 4s 36ms/step - loss: 0.2595 - accuracy: 0.9122 - val_loss: 0.5149 - val_accuracy: 0.8583
Epoch 7/40
105/105 [==============================] - 4s 36ms/step - loss: 0.2284 - accuracy: 0.9223 - val_loss: 1.3862 - val_accuracy: 0.7264
Epoch 8/40

In [32]:
load_model_1 = keras.models.load_model("models/transfer_1.h5")
load_model_1.evaluate(test)

 1/23 [>.............................] - ETA: 4s - loss: 0.3194 - accuracy: 0.8750

2022-08-02 06:33:15.189082: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 1s 21ms/step - loss: 0.3787 - accuracy: 0.8958


[0.37866339087486267, 0.8958333730697632]

# compare with untrained

In [22]:
inputs = keras.Input(shape=(100, 100, 3))
x = keras.layers.Conv2D(32, 3, activation='relu')(inputs)
x = keras.layers.MaxPooling2D(2)(x)
x = keras.layers.Conv2D(64, 3, activation='relu')(x)
x = keras.layers.MaxPooling2D(2)(x)
x = keras.layers.Conv2D(128, 3, activation='relu')(x)
x = keras.layers.MaxPooling2D(2)(x)
x = keras.layers.Conv2D(256, 3, activation='relu')(x)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(6, activation='softmax')(x)

umodel = keras.Model(inputs, outputs)

umodel.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

umodel.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 49, 49, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 21, 21, 128)       7385

In [23]:
callbacks_list = [
    # keras.callbacks.EarlyStopping(
    #     monitor="val_accuracy",
    #     patience=7
    # ),
    keras.callbacks.ModelCheckpoint(
        filepath="models/zenodo_from_vtid_spec.h5",
        monitor="val_loss",
        save_best_only=True
    )
]

history = model.fit(
  train.cache(),
  epochs=40,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Epoch 1/40
105/105 [==============================] - 4s 32ms/step - loss: 0.4972 - accuracy: 0.9443 - val_loss: 0.7289 - val_accuracy: 0.9125
Epoch 2/40
105/105 [==============================] - 4s 38ms/step - loss: 0.2194 - accuracy: 0.9625 - val_loss: 1.3313 - val_accuracy: 0.8653
Epoch 3/40
105/105 [==============================] - 3s 30ms/step - loss: 0.1397 - accuracy: 0.9702 - val_loss: 0.6823 - val_accuracy: 0.9264
Epoch 4/40
105/105 [==============================] - 3s 32ms/step - loss: 0.2336 - accuracy: 0.9622 - val_loss: 1.9846 - val_accuracy: 0.8528
Epoch 5/40
105/105 [==============================] - 4s 34ms/step - loss: 0.1712 - accuracy: 0.9702 - val_loss: 0.8968 - val_accuracy: 0.9153
Epoch 6/40
105/105 [==============================] - 4s 37ms/step - loss: 0.1775 - accuracy: 0.9747 - val_loss: 1.3534 - val_accuracy: 0.8556
Epoch 7/40
105/105 [==============================] - 4s 35ms/step - loss: 0.1551 - accuracy: 0.9714 - val_loss: 3.7150 - val_accuracy: 0.7958

In [34]:

load_model_2 = keras.models.load_model("models/zenodo_from_vtid_spec.h5")
load_model_2.evaluate(test)


2022-08-02 06:34:11.649794: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 1s 27ms/step - loss: 0.6121 - accuracy: 0.9431


[0.612091064453125, 0.9430555701255798]

# All Trainable Layers Transfer Learning - start from nonrandom weights

In [30]:
vtid_allTrainable = keras.models.load_model('models/vtid2_1.h5')
x = vtid_allTrainable.layers[-2].output
outputs = keras.layers.Dense(6, activation='softmax', name='classifier')(x)
model = keras.Model(vtid_allTrainable.inputs, outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 128)       738

In [31]:
callbacks_list = [
    # keras.callbacks.EarlyStopping(
    #     monitor="val_accuracy",
    #     patience=7
    # ),
    keras.callbacks.ModelCheckpoint(
        filepath="models/vtid_all_trainable.h5",
        monitor="val_loss",
        save_best_only=True
    )
]

history = model.fit(
  train.cache(),
  epochs=40,
  callbacks=callbacks_list,
  validation_data=val.cache()
)

Epoch 1/40


2022-08-02 06:27:56.319934: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - ETA: 0s - loss: 1.8264 - accuracy: 0.3074

2022-08-02 06:28:02.053414: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


105/105 [==============================] - 7s 57ms/step - loss: 1.8264 - accuracy: 0.3074 - val_loss: 1.2334 - val_accuracy: 0.5389
Epoch 2/40
105/105 [==============================] - 7s 62ms/step - loss: 1.2262 - accuracy: 0.5592 - val_loss: 1.2952 - val_accuracy: 0.5083
Epoch 3/40
105/105 [==============================] - 7s 67ms/step - loss: 0.8277 - accuracy: 0.7095 - val_loss: 1.8692 - val_accuracy: 0.5333
Epoch 4/40
105/105 [==============================] - 7s 64ms/step - loss: 0.6540 - accuracy: 0.7702 - val_loss: 0.7829 - val_accuracy: 0.7431
Epoch 5/40
105/105 [==============================] - 7s 66ms/step - loss: 0.5259 - accuracy: 0.8262 - val_loss: 0.4815 - val_accuracy: 0.8111
Epoch 6/40
105/105 [==============================] - 7s 67ms/step - loss: 0.3914 - accuracy: 0.8693 - val_loss: 0.5302 - val_accuracy: 0.8236
Epoch 7/40
105/105 [==============================] - 7s 66ms/step - loss: 0.3299 - accuracy: 0.8949 - val_loss: 0.5610 - val_accuracy: 0.8278
Epoch 8/40

In [35]:
load_model_3 = keras.models.load_model("models/vtid_all_trainable.h5")
load_model_3.evaluate(test)

 1/23 [>.............................] - ETA: 4s - loss: 0.7403 - accuracy: 0.9375

2022-08-02 06:34:20.280480: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 1s 23ms/step - loss: 0.3436 - accuracy: 0.9292


[0.34361395239830017, 0.9291666746139526]